# Import other python script functions

In [1]:
from data_cleaning import clean_raw_data

ModuleNotFoundError: No module named 'data_cleaning'

# Import resale flat price data

In [ ]:
df_raw = pd.read_csv('data/resale_flat_prices.csv')
df_raw